In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt 
import scipy.stats
from scipy.stats import chi2_contingency
from scipy.stats import shapiro
from scipy.stats import ranksums
nltk.download('punkt')
tokenizer = RegexpTokenizer(r'\w+')
from sklearn.externals import joblib
import numpy as np
import matplotlib.pyplot as plt
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesClassifier


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
all_data = pd.read_csv('complete_dataset.csv', sep = ',')
all_data.columns
all_data['country-source'].value_counts()
all_data = all_data.rename(columns = {'country-source':'source'})
all_data.source = all_data.source.replace({'USA': 0, 'RU': 1})

featuress = all_data['ner'].tolist()
clean_ner = []
import re
for x in featuress:
  x = re.sub(r'[^\w ]', '', x)
  clean_ner.append(x)
y = np.array(all_data['source'].values)
featuress = all_data['ner'].tolist()
clean_ner = []
import re
for x in featuress:
  x = re.sub(r'[^\w ]', '', x)
  clean_ner.append(x)
  
loaded_vec = joblib.load('gen_1_fit.pkl')
count = loaded_vec.transform(clean_ner)

#https://stackoverflow.com/questions/47665176/converting-a-pandas-dataframe-to-a-scipy-sparse-matrix
from scipy.sparse import csr_matrix
features_n = all_data[['article_len','ratio_not_stop_w','ratio_not_stop_w_t','avg_word_len', 'title_len', 'ratio_unique_words','A','B','D', 'E', 'F']]


m = scipy.sparse.hstack((t2, count))
y = np.array(all_data['source'].values)
X = np.array(features_n.values)
xg_train, xg_test, yg_train, yg_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

#Univariate selection

In [107]:
from scipy.stats import zscore
np.set_printoptions(suppress=True)
features_n = all_data[['article_len','ratio_not_stop_w','ratio_not_stop_w_t','avg_word_len', 'title_len', 'ratio_unique_words','A','B','D', 'E', 'F']]

test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(X, y)
# summarize scores
np.set_printoptions(precision=4)
print(fit.scores_)
features = fit.transform(X)
# summarize selected features
print(features[0:4,:])
#most relevenat feature identified by this analysis are article lenth, decisively most relevant, followed by title length, followed by frame A (human interest) followed by Frame E (anti-west)

NameError: ignored

#Recursive feature variation

In [127]:
X[1]

array([91.    ,  0.1648,  0.    ,  7.0989,  6.    ,  0.7912,  0.    ,
        1.    ,  0.    ,  0.    ,  0.    ])

In [128]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)
X_new[1]
#0, 3, 4, 6,7,8,9, 10

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


array([91.    ,  7.0989,  6.    ,  0.    ,  1.    ,  0.    ,  0.    ,
        0.    ])

#Variance threshold analysis

In [118]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
m = sel.fit_transform(X)
m


array([[ 242.    ,    6.6901,    7.    ,    1.    ],
       [  91.    ,    7.0989,    6.    ,    1.    ],
       [ 105.    ,    5.9048,    7.    ,    1.    ],
       ...,
       [  97.    ,    6.3711,    7.    ,    0.    ],
       [1850.    ,    6.1108,    7.    ,    1.    ],
       [1191.    ,    6.4467,    7.    ,    1.    ]])

#Feature Importance with Random Forests 

In [123]:
model = ExtraTreesClassifier(n_estimators=5000)
model.fit(X, y)
print(model.feature_importances_)

[0.1696 0.1718 0.0928 0.2098 0.1278 0.1653 0.0219 0.0142 0.0099 0.0104
 0.0064]
